I am going to do some mycobiome work with samples from C. jacchus
This uses some ITS data
The commands are based on the atacama soils tutorial for Qiime2
https://docs.qiime2.org/2023.9/tutorials/atacama-soils/
Let's activate the environment

In [ ]:
conda activate qiime2-amplicon-2023.9

Time to make the relevant folders for the data
We make a project folder, move into it, and make a folder for the sequences

In [ ]:
mkdir CjacchusITS
cd CjacchusITS
mkdir emp-paired-end-sequences

I downloaded the sequences manually and put them into the folder emp-paired-end-sequences
The functions (tools?) in qiime require these to have specific names

These are also .fastq files, not .fastq.gz files
I am going to make them fastq.gz by zipping and then rename them

ITS3_ITS4_pgarber_I1.fastq
renamed to barcodes.fastq.gz after zipping

ITS3_ITS4_pgarber_R1.fastq
renamed forward.fastq.gz after zipping

ITS3_ITS4_pgarber_R2.fastq
Renamed reverse.fastq.gz after zipping

I found the metadata file, it was .txt, I used Google sheets to make it .tsv (just in case that matters)
called Cjacchus_ITS_metadata.tsv

This code zips and renames the files
It appears to get ride of the unzipped files, FYI

In [ ]:
gzip ITS3_ITS4_pgarber_I1.fastq
mv ITS3_ITS4_pgarber_I1.fastq.gz barcodes.fastq.gz
gzip ITS3_ITS4_pgarber_R1.fastq
mv ITS3_ITS4_pgarber_R1.fastq.gz forward.fastq.gz
gzip ITS3_ITS4_pgarber_R2.fastq
mv ITS3_ITS4_pgarber_R2.fastq.gz reverse.fastq.gz

Let's pull these data in as an artifact

In [ ]:
qiime tools import --type EMPPairedEndSequences --input-path emp-paired-end-sequences --output-path emp-paired-end-sequences.qza

Now it is time to demultiplex the sequence reads. We are going to use the metadata and the column is called Barcode

The function/tool needs a 12nt barcode, but these are only 10nt
We used this flag to suppress that error correction
--p-no-golay-error-correction

We also need the flag below this line or we lose many samples
--p-rev-comp-barcodes 
It may be important that the following line was already in the code
--p-rev-comp-mapping-barcodes flag

In [ ]:
qiime demux emp-paired --m-barcodes-file Cjacchus_ITS_metadata.tsv --m-barcodes-column Barcode --p-rev-comp-mapping-barcodes --p-rev-comp-barcodes --p-no-golay-error-correction --i-seqs emp-paired-end-sequences.qza --o-per-sample-sequences demux-full.qza --o-error-correction-details demux-details.qza

Let's summarize these

In [ ]:
qiime demux summarize --i-demux-full.qza --o-visualization demux-full.qzv

I have variable primer lengths and need to do cutadapt to trim the primers off
These should be the same ITS primers from this code
https://github.com/Mallott-Lab/MultiSpecies_ITS/blob/main/ITS.ipynb

In [ ]:
qiime cutadapt trim-paired \
  --i-demultiplexed-sequences demux-full.qza \
  --p-adapter-f GCATATCAATAAGCGGAGGA \
  --p-front-f GCATCGATGAAGAACGCAGC \
  --p-adapter-r GCTGCGTTCTTCATCGATGC \
  --p-front-r TCCTCCGCTTATTGATATGC \
  --o-trimmed-sequences demux-trimmed.qza

In [ ]:
qiime demux summarize --i-data demux-trimmed.qza --o-visualization demux-trimmed.qzv

Due to the way that this sequencing was done, we are using 230 for truncation length
We don't need to truncate really, but the 230 accounts for the cut adapt that we did

In [ ]:
qiime dada2 denoise-paired --i-demultiplexed-seqs demux-trimmed.qza --p-n-threads 6 --p-trim-left-f 0 --p-trim-left-r 0 --p-trunc-len-f 230 --p-trunc-len-r 230 --o-table table-trimmed.qza --o-representative-sequences rep-seqs-trimmed.qza --o-denoising-stats denoising-stats-trimmed.qza

I am going to look at summaries of the artifacts produced by the above code

In [ ]:
qiime feature-table summarize --i-table table-trimmed.qza --o-visualization table-trimmed.qzv --m-sample-metadata-file Cjacchus_ITS_metadata.tsv
qiime feature-table tabulate-seqs --i-data rep-seqs-trimmed.qza --o-visualization rep-seqs-trimmed.qzv
qiime metadata tabulate --m-input-file denoising-stats-trimmed.qza --o-visualization denoising-stats-trimmed.qzv

Now it is time to assign taxonomy
Liz built a classifier already, called classifier.qza
I just need to bring it in and use it
I downloaded it and dropped it into the appropriate folder

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier classifier.qza \
  --i-reads rep-seqs-trimmed.qza \
  --o-classification taxonomy.qza

Let's visualize this

In [ ]:
qiime metadata tabulate \
  --m-input-file  taxonomy.qza \
  --o-visualization taxonomy.qzv

I am going to make a bar plot now

In [ ]:
qiime taxa barplot \
  --i-table table-trimmed.qza  \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file Cjacchus_ITS_metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

I have an interactive bar plot, there is a lot of "fungi" and "fungi_unidentified" stuff here
Two samples have no reads, but not too many, I should make some rarefaction curves to see what depth I might want to go to

In [ ]:
qiime diversity alpha-rarefaction --i-table table-trimmed.qza --p-max-depth 9000 --m-metadata-file Cjacchus_ITS_metadata.tsv --o-visualization alpha-rarefaction.qzv